# 02 - Exploratory Data Analysis (Drivers & Visuals)

In [ ]:
import os
import pandas as pd

filepath_train = r"../data/coral_students.csv"
df = pd.read_csv(filepath_train)

df.head()

In [ ]:
df.columns

In [ ]:
df['Percent_Bleached'].describe()